In [ ]:
# Check NVIDIA GPU status
!nvidia-smi


Thu Dec 12 16:29:27 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.104.05             Driver Version: 535.104.05   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  Tesla T4                       Off | 00000000:00:04.0 Off |                    0 |
| N/A   37C    P8               9W /  70W |      0MiB / 15360MiB |      0%      Default |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
from google.colab import drive

# Mount Google Drive to /content/drive
drive.mount('/content/drive')

Mounted at /content/drive


Step 1: Set Up the Environment


In [ ]:
# Install necessary libraries
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 898.7/898.7 kB 14.4 MB/s eta 0:00:00


In [ ]:
!pip install roboflow



   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 80.9/80.9 kB 3.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 6.0 MB/s eta 0:00:00
  Attempting uninstall: idna
    Found existing installation: idna 3.10
    Uninstalling idna-3.10:
      Successfully uninstalled idna-3.10


Download Roboflow project

DO NOT SHARE API KEY

In [ ]:
# USE ROBIFLOW PROJECT FOR YOLOv11
 # ENTER YOUR API KEY

from roboflow import Roboflow
rf = Roboflow(api_key="YOUR_API_KEY")
project = rf.workspace("research-plgpi").project("bccd_rbc_wbc_platelets")
version = project.version(1)
dataset = version.download("yolov11")


# 874 Total Images
# Dataset Split Train Set: 88%:: 765 Images
#                Valid Set: 8%:: 73 Images
#                 Test Set: 4%:: 36 Images

#### Preprocessing
# Auto-Orient: Applied
# Resize: Stretch to 640x480

#### Augmentations
# Outputs per training example: 3
# Flip: Horizontal, Vertical
# Crop: 0% Minimum Zoom, 20% Maximum Zoom
# Rotation: Between -15° and +15°
# Hue: Between -15° and +15°
# Saturation: Between -25% and +25%
# Brightness: Between -15% and +15%
# Blur: Up to 2.5px
# Noise: Up to 0.1% of pixels


loading Roboflow workspace...
loading Roboflow project...


Read https://docs.ultralytics.com/modes/ to know about modes

and also

Read https://docs.ultralytics.com/models/ to know about various models

We use Yolov11 so check for details at https://docs.ultralytics.com/models/yolo11/

and also

Read https://docs.ultralytics.com/modes/train/ to know about how to train and training parameters

Now copy code https://docs.ultralytics.com/tasks/detect/#how-do-i-train-a-yolo11-model-on-my-custom-dataset

Check the directory at /content/ and see that the dataset directory should be downlaoded and ready to use

Then modify the data.yaml from /content/Kickboard-6/valid directory

Select path of imgaes directory from each of train, valid and test directories to the data.yaml files in respective positions

In [ ]:
# copy code from https://docs.ultralytics.com/tasks/detect/#faq

# Check the directory at /content/ and see that the dataset directory is already prepared

from ultralytics import YOLO

# Load a pretrained model (largest model) in "/content/" directory i.e. local folder
model = YOLO("yolo11n.pt")


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


100%|██████████| 5.35M/5.35M [00:00<00:00, 16.5MB/s]


After traning is complete. Go to /content/runs/detect/train/weights and find the best.pt weight file. Download it and use it as a finetuned weight file for detecting objects.

Also various curves, results, plots are available here in /content/runs/detect/train

Now start prediction with our own finetuned model.

Go to https://docs.ultralytics.com/modes/predict/#key-features-of-predict-mode to know how rpediction takes place.

Repalce the pretrained YOLO11n.pt with /content/runs/detect/train/weights/best.pt

In [ ]:
#### Modiy data.yaml as per train, valid and test directory locations

from ultralytics import YOLO
import os

print("torch.cuda.is_available():", __import__('torch').cuda.is_available())
print("torch.cuda.device_count():", __import__('torch').cuda.device_count())

device_to_use = 0 if __import__('torch').cuda.is_available() else 'cpu'

model = YOLO("/content/yolo11n.pt")  # Pretrained large model

results = model.train(data="/content/bccd_rbc_wbc_platelets-1/data.yaml", epochs=200, imgsz=640)


torch.cuda.is_available(): True
torch.cuda.device_count(): 1
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=/content/yolo11n.pt, data=/content/bccd_rbc_wbc_platelets-1/data.yaml, epochs=200, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_t

100%|██████████| 755k/755k [00:00<00:00, 3.12MB/s]


Overriding model.yaml nc=80 with nc=3

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/bccd_rbc_wbc_platelets-1/train/labels... 765 images, 0 backgrounds, 0 corrupt: 100%|██████████| 765/765 [00:00<00:00, 1237.85it/s]

train: New cache created: /content/bccd_rbc_wbc_platelets-1/train/labels.cache


...

    149/200      3.02G     0.6919      0.436     0.9858        270        640: 100%|██████████| 48/48 [00:08<00:00,  5.98it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  7.41it/s]

                   all         73       1003       0.89        0.9      0.919      0.626
EarlyStopping: Training stopped early as no improvement observed in last 100 epochs. Best results observed at epoch 49, best model saved as best.pt.
To update EarlyStopping(patience=100) pass a new patience value, i.e. `patience=300` or use `patience=0` to disable EarlyStopping.



149 epochs completed in 0.378 hours.
Optimizer stripped from runs/detect/train/weights/last.pt, 5.5MB
Optimizer stripped from runs/detect/train/weights/best.pt, 5.5MB

Validating runs/detect/train/weights/best.pt...
Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 3/3 [00:00<00:00,  3.34it/s]


                   all         73       1003        0.9      0.915      0.948      0.654
             Platelets         43         73      0.883      0.959      0.962      0.501
                   RBC         70        855      0.821      0.785      0.888       0.63
                   WBC         73         75      0.998          1      0.995      0.829
Speed: 0.1ms preprocess, 1.5ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/train


 Create the Folder to save the finetuned model currently existing in /content/YOLO_BCCD_Training/yolovn11_bccd/weights as file name "best.pt"

In [ ]:
import os

# Define the source path
source_file = "/content/runs/detect/train/weights/best.pt"

# Check if the source file exists
if os.path.exists(source_file):
    print(f"Source file '{source_file}' exists.")
else:
    print(f"Source file '{source_file}' does NOT exist. Please check the path.")


Source file '/content/runs/detect/train/weights/best.pt' exists.


In [ ]:
import os

# Define the source path
source_file = "/content/runs/detect/train/weights/best.pt"

# Define the destination directory in Google Drive
destination_dir = "/content/drive/MyDrive/internship/yolov11bccd"

# Define the destination file path
destination_file = os.path.join(destination_dir, "best.pt")


In [ ]:
### Permanently saves the best.pt model to mydrive

import shutil

# Define the source of best.pt model
source_file = "/content/runs/detect/train/weights/best.pt"

# Define the destination to save above fine tuned model
destination_dir = "/content/drive/MyDrive/internship/yolov11bccd"

destination_file = os.path.join(destination_dir, "best.pt")

# Check if the source file exists
if os.path.exists(source_file):
    # Copy the file
    shutil.copy(source_file, destination_file)
    print(f"'best.pt' has been successfully copied to '{destination_dir}'.")
else:
    print(f"Source file '{source_file}' does not exist. Please check the path.")


'best.pt' has been successfully copied to '/content/drive/MyDrive/internship/yolov11bccd'.


In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [ ]:
!pip install ultralytics gradio


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 41.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 29.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 10.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 118.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 6.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 16.8 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:
# First run the code to get the validation metrics of best.pt

from ultralytics import YOLO

model_path = "/content/runs/detect/train/weights/best.pt"  # your trained model
data_yaml_path = "/content/bccd_rbc_wbc_platelets-1/data.yaml"  # dataset configuration file

model = YOLO(model_path)
metrics = model.val(data=data_yaml_path)  # ensure data.yaml points to the correct valid set

# Extract overall metrics
overall_precision = metrics.box.mp       # mean precision over all classes
overall_recall = metrics.box.mr          # mean recall over all classes
overall_map50 = metrics.box.map50        # mean AP at IoU=0.5 over all classes
overall_map = metrics.box.map            # mean AP at IoU=0.5:0.95 over all classes
overall_map75 = metrics.box.map75        # mean AP at IoU=0.75 over all classes

# Extract per-class metrics
class_names = model.names  # or load from data.yaml if needed, same as model.names
class_metrics = []
for i, cname in enumerate(class_names):
    p, r, ap50, ap = metrics.box.class_result(i)
    class_metrics.append((cname, p, r, ap50, ap))

print("Overall Metrics:")
print(f"Precision: {overall_precision}")
print(f"Recall: {overall_recall}")
print(f"mAP50: {overall_map50}")
print(f"mAP50-95: {overall_map}")
print(f"mAP75: {overall_map75}")
print("\nPer-Class Metrics:")
for (cname, p, r, ap50, ap) in class_metrics:
    print(f"{cname}: Precision={p}, Recall={r}, mAP50={ap50}, mAP50-95={ap}")


Ultralytics 8.3.49 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
YOLO11n summary (fused): 238 layers, 2,582,737 parameters, 0 gradients, 6.3 GFLOPs


val: Scanning /content/bccd_rbc_wbc_platelets-1/valid/labels.cache... 73 images, 0 backgrounds, 0 corrupt: 100%|██████████| 73/73 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.15it/s]


                   all         73       1003        0.9      0.915      0.948      0.653
             Platelets         43         73      0.882      0.959      0.961      0.502
                   RBC         70        855       0.82      0.787      0.888       0.63
                   WBC         73         75      0.998          1      0.995      0.826
Speed: 0.2ms preprocess, 3.8ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to runs/detect/val3
Overall Metrics:
Precision: 0.8998657967724281
Recall: 0.9152413015416975
mAP50: 0.9482967626275897
mAP50-95: 0.6529025986330599
mAP75: 0.7199225312247104

Per-Class Metrics:
0: Precision=0.8820047185253768, Recall=0.958904109589041, mAP50=0.961433378998409, mAP50-95=0.5023530432704303
1: Precision=0.8196728808767741, Recall=0.7868197950360514, mAP50=0.8884569088843599, mAP50-95=0.6302822447945686
2: Precision=0.9979197909151334, Recall=1.0, mAP50=0.995, mAP50-95=0.8260725078341811


In [ ]:
############## Use below for production with manual metrics input from above


import os
import torch
import cv2
import numpy as np
from ultralytics import YOLO
from PIL import Image
import yaml
import gradio as gr
import pandas as pd

model_path = "best.pt"   #### Use exact path or keep on local directory, I keep in local directory
data_yaml_path = "data.yaml"  #### Use exact path  or keep on local directory, I keep in local directory

if not os.path.exists(model_path):
    raise FileNotFoundError(f"Model file not found at {model_path}.")
if not os.path.exists(data_yaml_path):
    raise FileNotFoundError(f"data.yaml not found at {data_yaml_path}.")

# Load the YOLO model
model = YOLO(model_path)

# Load class names
with open(data_yaml_path, 'r') as stream:
    data_dict = yaml.safe_load(stream)
class_names = data_dict['names']  # e.g., ['Platelets', 'RBC', 'WBC'] if those are your classes

##################################
# Hardcoded metrics from your provided values:
overall_precision = 0.8998657967724281
overall_recall = 0.9152413015416975
overall_map50 = 0.9482967626275897
overall_map = 0.6529025986330599
overall_map75 = 0.7199225312247104

# Per-Class Metrics (index as per data.yaml order)
# Here we assume the class order matches the indices:
# class_names[0], class_names[1], class_names[2], etc.
class0_precision = 0.8820047185253768
class0_recall = 0.958904109589041
class0_map50 = 0.961433378998409
class0_map = 0.5023530432704303

class1_precision = 0.8196728808767741
class1_recall = 0.7868197950360514
class1_map50 = 0.8884569088843599
class1_map = 0.6302822447945686

class2_precision = 0.9979197909151334
class2_recall = 1.0
class2_map50 = 0.995
class2_map = 0.8260725078341811

# Construct the metrics DataFrame
metrics_data = [
    ["Overall", overall_precision, overall_recall, overall_map50, overall_map],
    [class_names[0], class0_precision, class0_recall, class0_map50, class0_map],
    [class_names[1], class1_precision, class1_recall, class1_map50, class1_map],
    [class_names[2], class2_precision, class2_recall, class2_map50, class2_map]
]
metrics_df = pd.DataFrame(metrics_data, columns=["Class", "Precision", "Recall", "mAP50", "mAP50-95"])
##################################

def run_inference(img: np.ndarray, model):
    img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    results = model.predict(img_rgb, conf=0.25, iou=0.6)
    detections = []
    res = results[0]
    boxes = res.boxes
    if boxes is not None and len(boxes) > 0:
        for i in range(len(boxes)):
            xyxy = boxes.xyxy[i].tolist()
            conf = float(boxes.conf[i])
            cls_idx = int(boxes.cls[i])
            class_name = class_names[cls_idx]
            detections.append([class_name, conf, *xyxy])
    return detections

def draw_boxes(image: np.ndarray, detections):
    # Define a color palette for classes (BGR)
    palette = [
        (0, 255, 0),    # Green
        (255, 0, 0),    # Blue
        (0, 0, 255),    # Red
        (255, 255, 0),  # Cyan
        (255, 0, 255),  # Magenta
        (0, 255, 255),  # Yellow
        (128, 0, 128),  # Purple
        (128, 128, 0),  # Olive
        (0, 128, 128),  # Teal
    ]
    num_colors = len(palette)

    for det in detections:
        class_name, conf, x1, y1, x2, y2 = det
        cls_idx = class_names.index(class_name)
        color = palette[cls_idx % num_colors]

        cv2.rectangle(image, (int(x1), int(y1)), (int(x2), int(y2)), color, 2)

        # Text settings
        label = f"{class_name} {conf:.2f}"
        font = cv2.FONT_HERSHEY_SIMPLEX
        font_scale = 0.8
        thickness = 2

        (tw, th), _ = cv2.getTextSize(label, font, font_scale, thickness)
        # Draw filled rectangle behind text
        cv2.rectangle(image, (int(x1), int(y1)-th-8), (int(x1)+tw, int(y1)), color, -1)
        # Put text in white for visibility
        cv2.putText(image, label, (int(x1), int(y1)-5), font, font_scale, (255, 255, 255), thickness, cv2.LINE_AA)

    return image

def process_image(image):
    img = np.array(image)
    img_bgr = cv2.cvtColor(img, cv2.COLOR_RGB2BGR)

    detections = run_inference(img_bgr, model)

    annotated_img = draw_boxes(img_bgr.copy(), detections)
    annotated_img_rgb = cv2.cvtColor(annotated_img, cv2.COLOR_BGR2RGB)

    det_df = pd.DataFrame(detections, columns=["Class", "Confidence", "x1", "y1", "x2", "y2"])

    # Return annotated image, detection results, and hardcoded metrics table
    return Image.fromarray(annotated_img_rgb), det_df, metrics_df

with gr.Blocks() as demo:
    gr.Markdown("# YOLOn11 Cell Detection Web App")
    gr.Markdown("Upload an image and the model will return bounding boxes, classes, and confidence scores.")
    gr.Markdown("Metrics shown below are pre-computed and hardcoded into the code.")

    with gr.Row():
        with gr.Column():
            input_image = gr.Image(type="pil", label="Upload Image")
            submit_btn = gr.Button("Run Inference")
        with gr.Column():
            output_image = gr.Image(type="pil", label="Annotated Image")
            det_results = gr.DataFrame(label="Detection Results")
            metrics_table = gr.DataFrame(value=metrics_df, label="Validation Metrics")

    submit_btn.click(fn=process_image, inputs=input_image, outputs=[output_image, det_results, metrics_table])

demo.launch()


FileNotFoundError: Model file not found at best.pt.